#### Hyperparameter tuning with optuna

In [1]:
import os
import numpy as np
import platform

import optuna

import torch
from torch_geometric.datasets import AMiner
from torch_geometric.nn import MetaPath2Vec

from utils import *

In [2]:
EPS = 1e-15

path = os.path.join(os.getcwd(), 'data/AMiner')
dataset = AMiner(path)
data = dataset[0]

metapath = [
    ('author', 'writes', 'paper'),
    ('paper', 'published_in', 'venue'),
    ('venue', 'publishes', 'paper'),
    ('paper', 'written_by', 'author'),
]

device = 'cuda' if torch.cuda.is_available() else 'cpu'
optuna_version = optuna.__version__

print(f"using {device} device and optuna version is {optuna_version}")

using cuda device and optuna version is 2.10.0


In [3]:
@torch.no_grad()
def test(model, train_ratio=0.1):
    # returns train/test score after fitting Logistic Regression model
    model.eval()

    # changing device type is needed (to(device))
    z = model('author', batch=data['author'].y_index.to(device))
    y = data['author'].y    # author id

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(
        z[train_perm], y[train_perm], z[test_perm], y[test_perm], max_iter=150)

##### Define objective function

In [4]:
# since model is too heavy, we only tune context_size and walk_length
# if you increase either embedding_dim, walk_length, walks_per_node or num_negative_samples too much,
# then you will encounter runtime error: out of memory

def objective(trial):
    params = {
        'context_size': trial.suggest_int('context_size', 3, 7),
        'walk_length': trial.suggest_categorical('walk_length', [15, 20, 25])
    }

    model = MetaPath2Vec(
        data.edge_index_dict,
        embedding_dim=32,
        metapath=metapath,
        walk_length=params['walk_length'],
        context_size=params['context_size'],
        walks_per_node=2,
        num_negative_samples=2,
        sparse=True).to(device)

    optimizer = torch.optim.SparseAdam(
        list(model.parameters()), lr=1e-2, betas=(0.9, 0.999), eps=1e-08)

    # train & evaluate
    num_workers = 0 if platform.system() == 'Windows' else os.cpu_count()
    loader = model.loader(batch_size=128, shuffle=True, num_workers=num_workers)

    for epoch in range(1, EPOCHS+1):
        model.train()
        total_loss = 0

        for i, (pos_rw, neg_rw) in enumerate(loader):
            optimizer.zero_grad()
            loss = model.loss(pos_rw.to(device), neg_rw.to(device))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        test_acc = test(model, train_ratio=0.1)
        
        # report after 1 epoch ends
        trial.report(test_acc, epoch)
    
        # handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return test_acc


In [5]:
# torch.cuda.empty_cache()
get_memory_stat(device)

,category,memory_stat
0,total,8192.0
1,reserved,0.0
2,allocated,0.0


##### Execute tuning process

In [6]:
EPOCHS = 2

# you can choose pruner
# ref: https://towardsdatascience.com/hyperparameter-tuning-of-neural-networks-with-optuna-and-pytorch-22e179efc837
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner()
    )

study.optimize(objective, n_trials=8, timeout=3000)

[I 2022-01-02 18:38:58,280] A new study created in memory with name: no-name-399f50b5-2355-491b-a823-7189616f45b3
[I 2022-01-02 18:41:03,769] Trial 0 finished with value: 0.3857466521929094 and parameters: {'context_size': 5, 'walk_length': 20}. Best is trial 0 with value: 0.3857466521929094.
[I 2022-01-02 18:43:24,423] Trial 1 finished with value: 0.3869042524919936 and parameters: {'context_size': 4, 'walk_length': 25}. Best is trial 1 with value: 0.3869042524919936.
[I 2022-01-02 18:45:26,722] Trial 2 finished with value: 0.30338136398646914 and parameters: {'context_size': 4, 'walk_length': 20}. Best is trial 1 with value: 0.3869042524919936.
[I 2022-01-02 18:47:47,208] Trial 3 finished with value: 0.4938358910144092 and parameters: {'context_size': 5, 'walk_length': 25}. Best is trial 3 with value: 0.4938358910144092.
[I 2022-01-02 18:49:50,042] Trial 4 finished with value: 0.3347807090639653 and parameters: {'context_size': 3, 'walk_length': 20}. Best is trial 3 with value: 0.493

In [7]:
# aparently more resources you use, performance goes up
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print(f"#pruned: {len(pruned_trials)}, #complete: {len(complete_trials)}")

best_trial = study.best_trial

print("Best trial: ")
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

#pruned: 2, #complete: 6
Best trial: 
context_size: 7
walk_length: 25


##### Visualization of tuning process

In [8]:
# parameter importance
optuna.visualization.plot_param_importances(study)

In [9]:
# performance over multiple iterations
optuna.visualization.plot_optimization_history(study)

In [10]:
# performance of individual hyperparameter
optuna.visualization.plot_slice(study, params=['context_size', 'walk_length'])

In [11]:
# if you have a reasonable amout of hyperparameters and search space,
# you can see the value combination of hyperparameters and their performance
# on your objective function in each line
optuna.visualization.plot_parallel_coordinate(study)

##### Create ini file containing best hyperparameter values

In [12]:
best_params = best_trial.params
best_params.update({
    "embedding_dim": 32,
    "walks_per_node": 2,
    "num_negative_samples": 2
})
# best_params = {k:str(v) for k, v in best_params.items()}
print(best_params)

{'context_size': 7, 'walk_length': 25, 'embedding_dim': 32, 'walks_per_node': 2, 'num_negative_samples': 2}


In [14]:
parser = ConfigParser()
_ = parser.read('config.ini')

if not parser.has_section('setting'):
    parser.add_section('setting')

for k, v in best_params.items():
    parser.set('setting', k, str(v))

with open('config.ini', 'w') as configfile:
    parser.write(configfile)